In [1]:
import numpy as np
from psy import EmDina, MlDina
from psy.utils import r4beta

In [2]:
attrs = np.random.binomial(1, 0.5, (5, 60))
skills = np.random.binomial(1, 0.7, (1000, 5))


In [3]:
print(attrs)
print(attrs.shape)

[[1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1
  1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 0]
 [0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 1 0
  0 1 0 0 1 0 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1]
 [0 0 1 1 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1
  0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1]
 [1 0 0 1 1 1 0 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0
  0 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1]
 [1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1
  0 0 1 1 1 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0 1]]
(5, 60)


In [4]:
print(skills)
print(skills.shape)

[[1 1 1 1 1]
 [1 1 1 0 1]
 [1 1 1 1 1]
 ...
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 0]]
(1000, 5)


In [5]:
g = r4beta(1, 2, 0, 0, (1, 60))
no_s = r4beta(2, 1, 1, 1, (1, 60))

In [6]:
print(g)

print(g.shape)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(1, 60)


In [7]:
print(no_s)
print(no_s.shape)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
(1, 60)


In [10]:
temp = EmDina(attrs=attrs)
yita = temp.get_yita(skills)
p_val = temp.get_p(yita, guess=g, no_slip=no_s)
score = np.random.binomial(1, p_val)

In [8]:
# 60 items, 5 attributes

print(attrs)
print(attrs.shape)

[[1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1
  1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 0]
 [0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 1 0
  0 1 0 0 1 0 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1]
 [0 0 1 1 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1
  0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1]
 [1 0 0 1 1 1 0 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0
  0 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1]
 [1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1
  0 0 1 1 1 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0 1]]
(5, 60)


In [11]:
print(score)
print(score.shape)

# 1000 students, 60 items, 5 attributes

[[1 1 1 ... 1 1 1]
 [0 1 1 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 0 1 0]]
(1000, 60)


In [12]:
# Estimate the parameters
em_dina = EmDina(attrs=attrs, score=score)
est_no_s, est_g = em_dina.em()

print(np.mean(np.abs(est_no_s - no_s)))
print(np.mean(np.abs(est_g - g)))

9.501269215839159e-11
5.435775604768265e-09


In [13]:

# Estimate the student's skill mastery
dina_est = MlDina(guess=est_g, no_slip=est_no_s, attrs=attrs, score=score)
est_skills = dina_est.solve()

TypeError: __init__() missing 2 required positional arguments: 'guess' and 'no_slip'

In [ ]:
print(est_skills)
print(est_skills.shape)